# BackTesting

Import Libraries

In [2]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover 

Creating a class SMAcross which is inheriting from Strategy(imported from Backtesting library)
SMA is Simple Moving Average
We want to define short term simple moving average window and long term SMA window

In [3]:
class SMAcross(Strategy):

    ShortTerm_SMA_Window = 50   # It could be in timestep or days
    LongTerm_SMA_Window = 100 

    def init(self):
        close = self.data.Close # Close Price, Data is provided in seconds
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.ShortTerm_SMA_Window ) # Defining short term SMA using I
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.LongTerm_SMA_Window ) # Defining long term SMA using I
    
    def next(self):
        """Buy when we have a cross over of short term sma to long term sma
        When long term sma crosses over short term we want to sell"""
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [4]:
df = yf.download('HDFC.NS', start='2018-01-01')

[*********************100%***********************]  1 of 1 completed


In [5]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,1710.000000,1711.050049,1677.500000,1683.750000,1568.358032,1387222
2018-01-02,1690.000000,1706.250000,1685.400024,1703.250000,1586.521606,1468475
2018-01-03,1705.199951,1711.800049,1695.000000,1700.800049,1584.239624,2133332
2018-01-04,1708.949951,1708.949951,1691.050049,1703.199951,1586.475098,1585554
2018-01-05,1716.000000,1731.000000,1705.849976,1725.199951,1606.967285,2302884
...,...,...,...,...,...,...
2022-06-13,2115.000000,2137.649902,2089.350098,2128.449951,2128.449951,3700786
2022-06-14,2094.000000,2132.000000,2090.050049,2109.500000,2109.500000,3631959
2022-06-15,2097.000000,2110.949951,2071.300049,2105.449951,2105.449951,4365287


In [7]:
bt = Backtest(df, SMAcross, cash=100000, commission=0.002, exclusive_orders=True)

In [8]:
output = bt.run()

In [9]:
output

Start                     2018-01-01 00:00:00
End                       2022-06-17 00:00:00
Duration                   1628 days 00:00:00
Exposure Time [%]                   82.486388
Equity Final [$]                 75333.679211
Equity Peak [$]                 108710.888959
Return [%]                         -24.666321
Buy & Hold Return [%]               21.912395
Return (Ann.) [%]                   -6.271755
Volatility (Ann.) [%]               30.550403
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -57.641684
Avg. Drawdown [%]                  -18.673387
Max. Drawdown Duration     1065 days 00:00:00
Avg. Drawdown Duration      268 days 00:00:00
# Trades                                   13
Win Rate [%]                        53.846154
Best Trade [%]                      21.755986
Worst Trade [%]                    -29.812986
Avg. Trade [%]                    

In [10]:
bt.plot()

Row(id='1449', ...)

### Optimization

In [12]:
optim = bt.optimize(ShortTerm_SMA_Window = range(50, 160, 10), 
                    LongTerm_SMA_Window = range(50,160, 10), 
                    constraint= lambda x: x.LongTerm_SMA_Window - x.ShortTerm_SMA_Window > 20, 
                    maximize='Return [%]' )
bt.plot()

Row(id='2511', ...)

In [13]:
optim

Start                     2018-01-01 00:00:00
End                       2022-06-17 00:00:00
Duration                   1628 days 00:00:00
Exposure Time [%]                   78.584392
Equity Final [$]                112265.598322
Equity Peak [$]                 155981.717453
Return [%]                          12.265598
Buy & Hold Return [%]               21.912395
Return (Ann.) [%]                    2.681019
Volatility (Ann.) [%]               31.618335
Sharpe Ratio                         0.084793
Sortino Ratio                        0.127164
Calmar Ratio                         0.059379
Max. Drawdown [%]                    -45.1513
Avg. Drawdown [%]                   -6.469932
Max. Drawdown Duration      487 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                    9
Win Rate [%]                        55.555556
Best Trade [%]                      44.945366
Worst Trade [%]                    -25.389261
Avg. Trade [%]                    